In [13]:
'''
Author  : Jiayao Yu, User Interfaces Group, Aalto University
Init    : August, 2017
Project : ELEC-D7010 Engineering for Humans course materials
Topic   : Multiple Resource Theory
===========================================================================
Usage explanation
------
where's reference from?
Multiple Resource Theory is to predict the relative differences in task
interference between different task configurations.

parameters shorthand explainations
------
      Task Analysis Shell: PC = Perception & Cognition, R = Responding,
                          V (under Modalities) = Visual, A = Auditory,
                          V (under Codes) = Verbal, S = Spatial
      Conflict Matrix: A and V in the first position = Auditory and Visual
                      V and S in the second position = Verbal and spatial
                      C = Cognitive, R = Response
Model limitations
------
1) assuming all tasks are alike
'''

import tkinter as tk
from tkinter import ttk
import numpy as np
import matplotlib as mpl
from IPython import display
%matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection, Line3DCollection
import matplotlib.pyplot as plt
import numpy as np
from itertools import product, combinations

class MRT(tk.Frame):
	tasks = ['driving', 
			'reading-map', 
			'listening-navigation']
	tasksAnalysis = ['S PC+R V',
					'S PC+R V',
					'V PC A']
	taskAnalysisValues = [[1, 0, 1, 1, 0, 1], 
						[1, 0, 1, 1, 0, 1], 
						[0, 1, 1, 0, 1, 0]]     # taskAnalysisValues[task][Perception&Cognition Responding Visual Auditory Verbal Spatial]
	maxTaskSelection = 2
	indexSelectedTask = []		# index of 2 selected tasks

	def __init__(self, parent=None):
		tk.Frame.__init__(self, parent)
		self.pack()
		self.makeWidgets()
		self.taskAnalysis()
		self.conflictMatrix()
		# self.cubeVis()

	def makeWidgets(self):
		self.frameTaskSelection = tk.Frame(self)
		self.frameTaskAnalysis = tk.Frame(self)
		self.frameConflictMatrix = tk.Frame(self)
		self.frameTaskSelection.grid(row=0, column=0, sticky=tk.NSEW, padx=10, pady=10)
		self.frameTaskAnalysis.grid(row=0, column=1, sticky=tk.NSEW, padx=10, pady=10)
		self.frameConflictMatrix.grid(row=0, column=2, sticky=tk.NSEW, padx=10, pady=10)
		self.btnsTask = []
		for i in range(len(self.tasks)):
			n = tk.Button(self.frameTaskSelection, text=self.tasks[i], width=20)
			self.btnsTask.append(n)
			self.btnsTask[-1].grid(row=i, column=0, sticky=tk.NSEW)
			self.btnsTask[-1].bind('<ButtonPress-1>', self.taskSelection)
	
	def taskSelection(self, event):
		for i in range(len(self.tasks)):
			if event.widget is self.btnsTask[i]:
				if self.btnsTask[i].cget('bg') == 'green':
					self.btnsTask[i].config(bg='SystemButtonFace', fg='black')
					if i in self.indexSelectedTask:
						self.indexSelectedTask.remove(i)
				else:
					if len(self.indexSelectedTask) < self.maxTaskSelection:
						self.btnsTask[i].config(bg='green', fg='white')
						self.indexSelectedTask.append(i)
		# hightlight associated cells according to tasks selection
		self.taskAnalysis()
		self.conflictMatrix()

		self.cubeVis()
		self.interference()

	def taskAnalysis(self):
		self.tableTaskAnalysisShell = ttk.Treeview(self.frameTaskAnalysis, columns=('codes', 'stages', 'modalities'), selectmode="browse")
		self.labelTaskAnalysisTitle = tk.Label(self.frameTaskAnalysis, text="Task Analysis Shell", anchor=tk.CENTER)
		self.tableTaskAnalysisShell.grid(row=0, column=0)
		self.labelTaskAnalysisTitle.grid(row=1, column=0)
		self.tableTaskAnalysisShell.column('#0', width=100)
		self.tableTaskAnalysisShell.column('codes', width=70, stretch=True)
		self.tableTaskAnalysisShell.column('stages', width=70, stretch=True)
		self.tableTaskAnalysisShell.column('modalities', width=70, stretch=True)
		self.tableTaskAnalysisShell.heading('codes', text='Codes')
		self.tableTaskAnalysisShell.heading('stages', text='Stages')
		self.tableTaskAnalysisShell.heading('modalities', text='Modalities')
		for i in range(len(self.tasks)):
			self.tableTaskAnalysisShell.insert('', i, self.tasks[i], text=self.tasks[i], tags=i, values=self.tasksAnalysis[i])
		# hightlight associated cells according to tasks selection
		for i in self.indexSelectedTask:
			self.tableTaskAnalysisShell.tag_configure(i, background='green', foreground='white')

	def cubeVis(self):
		# legend
		fig = plt.figure(figsize=(7,6), dpi=100, facecolor='w', edgecolor='k')
		ax = fig.add_subplot(111, projection='3d')
		# labelling dimensions and associated levels
		labelLevels = [['', 'Sptial', '\n\n<CODES>', 'Verbal', ''], 
						['', 'Perception&\nCognition', '\n\n<STAGES>', 'Responding', ''], 
						['', 'Auditory', '<MODALITIES>', 'Visual', '']]
		axisTicks = (-1, -0.5, 0, 0.5, 1)
		ax.set_xlim(-1,1)
		ax.set_xticks(axisTicks)
		ax.set_xticklabels(labelLevels[0])
		ax.set_ylim(-1,1)
		ax.set_yticks(axisTicks)
		ax.set_yticklabels(labelLevels[1])
		ax.set_zlim(-1,1)
		ax.set_zticks(axisTicks)
		ax.set_zticklabels(labelLevels[2])
		# determine associated cube vertices according to task selection
		r = 1
		x, y, z = ([0] for m in range(3))
		for i in range(len(self.indexSelectedTask)):
			for j in range(len(self.taskAnalysisValues[self.indexSelectedTask[i]])):
				if self.taskAnalysisValues[self.indexSelectedTask[i]][j]:
					if j < 2:
						x.append(np.sign(j-0.5))
					elif j < 4:
						y.append(np.sign(j-2.5))
					elif j < 6:
						z.append(np.sign(j-4.5))
		vertices = list(product(x, y, z))
		bufferVertices = [(1.0,1.0,1.0), (-1.0,1.0,1.0),(-1.0,-1.0,1.0),(1.0,-1.0,1.0),(1.0,1.0,-1.0),(-1.0,1.0,-1.0),(-1.0,-1.0,-1.0),(1.0,-1.0,-1.0)]
		bufferFaces = [[[[0,0,0],[1,0,0],[1,1,0],[0,1,0]],[[0,0,1],[1,0,1],[1,1,1],[0,1,1]],[[0,0,0],[1,0,0],[1,0,1],[0,0,1]],[[0,1,0],[1,1,0],[1,1,1],[0,1,1]],[[1,0,0],[1,1,0],[1,1,1],[1,0,1]],[[0,0,0],[0,1,0],[0,1,1],[0,0,1]]],
						[[[-1,0,0],[0,0,0],[0,1,0],[-1,1,0]],[[-1,0,1],[0,0,1],[0,1,1],[-1,1,1]],[[-1,0,0],[0,0,0],[0,0,1],[-1,0,1]],[[-1,1,0],[0,1,0],[0,1,1],[-1,1,1]],[[0,0,0],[0,1,0],[0,1,1],[0,0,1]],[[-1,0,0],[-1,1,0],[-1,1,1],[-1,0,1]]],
						[[[-1,-1,0],[0,-1,0],[0,0,0],[-1,0,0]],[[-1,-1,1],[0,-1,1],[0,0,1],[-1,0,1]],[[-1,-1,0],[0,-1,0],[0,-1,1],[-1,-1,1]],[[-1,0,0],[0,0,0],[0,0,1],[-1,0,1]],[[0,-1,0],[0,0,0],[0,0,1],[0,-1,1]],[[-1,-1,0],[-1,0,0],[-1,0,1],[-1,-1,1]]],
						[[[0,-1,0],[1,-1,0],[1,0,0],[0,0,0]],[[0,-1,1],[1,-1,1],[1,0,1],[0,0,1]],[[0,-1,0],[1,-1,0],[1,-1,1],[0,-1,1]],[[0,0,0],[1,0,0],[1,0,1],[0,0,1]],[[1,-1,0],[1,0,0],[1,0,1],[1,-1,1]],[[0,-1,0],[0,0,0],[0,0,1],[0,-1,1]]],
						[[[0,0,-1],[1,0,-1],[1,1,-1],[0,1,-1]],[[0,0,0],[1,0,0],[1,1,0],[0,1,0]],[[0,0,-1],[1,0,-1],[1,0,0],[0,0,0]],[[0,1,0],[0,1,-1],[1,1,-1], [1,1,0]],[[1,0,-1],[1,1,-1],[1,1,0],[1,0,0]],[[0,0,-1],[0,1,-1],[0,1,0],[0,0,0]]],
						[[[-1,0,-1],[0,0,-1],[0,1,-1],[-1,1,-1]],[[-1,0,0],[0,0,0],[0,1,0],[-1,1,0]],[[-1,0,-1],[0,0,-1],[0,0,0],[-1,0,0]],[[-1,1,-1],[0,1,-1], [0,1,0],[-1,1,0]],[[0,0,-1],[0,1,-1],[0,1,0],[0,0,0]],[[-1,0,-1],[-1,1,-1],[-1,1,0],[-1,0,0]]],
						[[[-1,-1,-1],[0,-1,-1],[0,0,-1],[-1,0,-1]],[[-1,-1,0],[0,-1,0],[0,0,0],[-1,0,0]],[[-1,-1,-1],[0,-1,-1],[0,-1,0],[-1,-1,0]],[[-1,0,-1], [0,0,-1],[0,0,0],[-1,0,0]],[[0,-1,-1],[0,0,-1],[0,0,0],[0,-1,0]],[[-1,-1,-1],[-1,0,-1],[-1,0,0],[-1,-1,0]]],
						[[[0,-1,-1],[1,-1,-1],[1,0,-1],[0,0,-1]],[[0,-1,0],[1,-1,0],[1,0,0],[0,0,0]],[[0,-1,0],[0,-1,-1],[1,-1,-1],[1,-1,0]],[[0,0,-1],[1,0,-1], [1,0,0],[0,0,0]],[[1,-1,-1],[1,0,-1],[1,0,0],[1,-1,0]],[[0,-1,-1],[0,0,-1],[0,0,0],[0,-1,0]]]]
		for i in range(len(bufferVertices)):
			if vertices.count(bufferVertices[i]) == 1:
				collection = Poly3DCollection(bufferFaces[i],alpha=0.5,linewidths=1)
				collection.set_facecolor('green')
				ax.add_collection3d(collection)
			if vertices.count(bufferVertices[i]) > 1:
				collection = Poly3DCollection(bufferFaces[i],alpha=0.5,linewidths=1)
				collection.set_facecolor('red')
				ax.add_collection3d(collection)
		display.clear_output(wait=True)
		plt.show()

	def conflictMatrix(self):
		self.tableConflictMatrix = tk.Canvas(self.frameConflictMatrix, relief=tk.GROOVE, bd=2, bg='white')
		self.tableConflictMatrix.grid(row=0, column=0)
		tk.Label(self.frameConflictMatrix, text="Conflict Matrix").grid(row=1, column=0)
		cellWidth = 40
		cellHeight = 25
		self.cell=[[]]
		for x in range(11):
			for y in range(11):
				self.cell[-1].append(self.tableConflictMatrix.create_rectangle((x-1)*cellWidth, (y-1)*cellHeight, x*cellWidth, y*cellHeight, outline=''))
			if x!= 10:
				self.cell.append([])
		self.textPerceptual = self.tableConflictMatrix.create_text(3*cellWidth, 0.5*cellHeight, text='Perceptual')
		self.textCognitive = self.tableConflictMatrix.create_text(6*cellWidth, 0.5*cellHeight, text='Cognitive')
		self.textResponse = self.tableConflictMatrix.create_text(8*cellWidth, 0.5*cellHeight, text='Response')
		levelsOfDimensions = ['VS', 'VV', 'AS', 'AV', 'CS', 'CV', 'RS', 'RV']
		self.textLevelsOfDimensions = [0 for m in range(2)]
		for m in range(2):
			self.textLevelsOfDimensions[m] = [0 for n in range(8)]
			for n in range(8):
				if m == 0:
					self.textLevelsOfDimensions[m][n] = self.tableConflictMatrix.create_text(0.5*cellWidth, (2.5+n)*cellHeight, text=levelsOfDimensions[n])
				elif m == 1:
					self.textLevelsOfDimensions[m][n] = self.tableConflictMatrix.create_text((1.5+n)*cellWidth, 1.5*cellHeight, text=levelsOfDimensions[n])
		self.valuesOfConflicts = ["0.8", '0.6', '0.6', '0.4', '0.7', '0.5', '0.4', '0.2',
									'', '0.8', '0.4', '0.6', '0.5', '0.7', '0.2', '0.4',
									'', '', '0.8', '0.4', '0.7', '0.5', '0.4', '0.2',
									'', '', '0', '0.8', '0.5', '0.7', '0.2', '0.4',
									'', '', '0', '', '0.8', '0.6', '0.6', '0.4',
									'', '', '', '', '', '0.8', '0.4', '0.6',
									'', '', '', '', '', '', '0.8', '0.6',
									'', '', '', '', '', '', '0.6', '1.0']
		self.valuesOfConflictsIntermd = [0.8, 0.6, 0.6, 0.4, 0.7, 0.5, 0.4, 0.2,
										0, 0.8, 0.4, 0.6, 0.5, 0.7, 0.2, 0.4,
										0, 0, 0.8, 0.4, 0.7, 0.5, 0.4, 0.2,
										0, 0, 0, 0.8, 0.5, 0.7, 0.2, 0.4,
										0, 0, 0, 0, 0.8, 0.6, 0.6, 0.4,
										0, 0, 0, 0, 0, 0.8, 0.4, 0.6,
										0, 0, 0, 0, 0, 0, 0.8, 0.6,
										0, 0, 0, 0, 0, 0, 0.6, 1.0]
		self.textValuesOfConflicts = [[0 for q in range(8)] for p in range(8)]
		for p in range(8):
			for q in range(8):
				self.textValuesOfConflicts[p][q] = self.tableConflictMatrix.create_text((1.5+q)*cellWidth, (2.5+p)*cellHeight, text=self.valuesOfConflicts[p*8+q])
		self.tableConflictMatrix.create_line(cellWidth, cellHeight, 9*cellWidth, cellHeight, width=2)
		self.tableConflictMatrix.create_line(10, 6*cellHeight, 9*cellWidth, 6*cellHeight, dash=(4, 4))
		self.tableConflictMatrix.create_line(10, 8*cellHeight, 9*cellWidth, 8*cellHeight)
		self.tableConflictMatrix.create_line(5*cellWidth, 10, 5*cellWidth, 10*cellHeight, dash=(4, 4))
		self.tableConflictMatrix.create_line(7*cellWidth, 10, 7*cellWidth, 10*cellHeight)
		# find index of cell that need to be highlighted
		indexHighlight = [[]]		
		if len(self.indexSelectedTask) == 2:
			for i in self.indexSelectedTask:
				if self.taskAnalysisValues[i][2]:
					if self.taskAnalysisValues[i][0]:
						if self.taskAnalysisValues[i][4]:
							# PC stages, S codes, A modalities ==> row no.5, 3
							indexHighlight[-1].append(3)
						if self.taskAnalysisValues[i][5]:
							# PC stages, S codes, V modalities ==> row no.5, 1
							indexHighlight[-1].append(1)
						indexHighlight[-1].append(5)
					if self.taskAnalysisValues[i][1]:
						if self.taskAnalysisValues[i][4]:
							# PC stages, V codes, A modalities ==> row no.6, 4
							indexHighlight[-1].append(4)
						if self.taskAnalysisValues[i][5]:
							# PC stages, V codes, V modalities ==> row no.6, 2
							indexHighlight[-1].append(2)
						indexHighlight[-1].append(6)
				if self.taskAnalysisValues[i][3]:
					if self.taskAnalysisValues[i][0]:
						# R in stages, S in codes ==> row no.7
						indexHighlight[-1].append(7)
					if self.taskAnalysisValues[i][1]:
						# R in srages, V in codes ==> row no.8
						indexHighlight[-1].append(8)
				if len(indexHighlight) != 2:
					indexHighlight.append([])
				print(indexHighlight)
			# highlight associated cells according to tasks selection
			index = list(product(indexHighlight[0], indexHighlight[1]))
			print(index)
			for i in list(index):
				tmp = sorted(list(i), reverse=True)
				print(tmp)
				self.tableConflictMatrix.itemconfigure(self.tmp[i[0]+2][tmp[1]+2], fill='green')
			self.tableConflictMatrix.itemconfigure(self.cell[3][3], fill='green')		



	def interference(self):
		# demand component: summing the average demand of both tasks
		demand = 0
		for i in range(len(self.indexSelectedTask)):
			demand += np.sum(self.taskAnalysisValues[self.indexSelectedTask[i]])
		demand /= 2
		# print(demand)
		# conflict component: degree of conflict between tasks on each resource channel
		self.conflictMatrix()
		# interference = demand + conflict


if __name__ == "__main__":
	root = tk.Tk()
	root.title("Multiple Resource Theory")
	MRT(root).mainloop()
	root.after(5, lambda: root.focus_force())

[[1, 5, 7], []]
[[1, 5, 7], [1, 5, 7]]
[(1, 1), (1, 5), (1, 7), (5, 1), (5, 5), (5, 7), (7, 1), (7, 5), (7, 7)]
[1, 1]


Exception in Tkinter callback
Traceback (most recent call last):
  File "D:\Anaconda3\lib\tkinter\__init__.py", line 1699, in __call__
    return self.func(*args)
  File "<ipython-input-13-078682d13954>", line 86, in taskSelection
    self.conflictMatrix()
  File "<ipython-input-13-078682d13954>", line 247, in conflictMatrix
    self.tableConflictMatrix.itemconfigure(self.tmp[i[0]+2][tmp[1]+2], fill='green')
AttributeError: 'MRT' object has no attribute 'tmp'


In [ ]:
list=[[]]
list[-1]